# Examen teórico-práctico

#### Mauricio Tumalan Castillo
#### A01369288

## Introducción

En este examen teórico-práctico, desarrollaremos un clasificador de texto utilizando redes neuronales. El objetivo principal es identificar y clasificar textos en una de las 20 categorías disponibles en el conjunto de datos "The 20 Newsgroups". Este conjunto de datos es ampliamente utilizado en tareas de clasificación de texto y procesamiento de lenguaje natural (NLP).

Para lograr este objetivo, seguiremos los siguientes pasos:

1. **Carga y exploración de datos**: Utilizaremos la biblioteca `sklearn` para cargar el conjunto de datos y exploraremos su estructura y contenido.
2. **Preprocesamiento de texto**: Aplicaremos técnicas de preprocesamiento de texto, como la eliminación de stopwords y la vectorización de texto utilizando TF-IDF.
3. **División de datos**: Dividiremos el conjunto de datos en subconjuntos de entrenamiento y prueba para evaluar el rendimiento del modelo.
4. **Construcción del modelo**: Crearemos un modelo de red neuronal utilizando `tensorflow` y `keras`.
5. **Entrenamiento y evaluación**: Entrenaremos el modelo con los datos de entrenamiento y evaluaremos su rendimiento utilizando métricas adecuadas.
6. **Visualización de resultados**: Utilizaremos bibliotecas de visualización como `plotly` para visualizar los resultados y las métricas de rendimiento.

A lo largo del notebook, se proporcionará una explicación detallada de cada paso y se documentará el uso de las librerías necesarias para completar el proyecto.

# Documentación de Uso de Librerías

En este notebook, utilizaremos varias librerías de Python para realizar diversas tareas de procesamiento y análisis de datos. A continuación, se describen las librerías que se importarán y su propósito:

1. **numpy**:
   - `np`: Biblioteca fundamental para la computación científica en Python. Proporciona soporte para matrices y operaciones matemáticas de alto nivel.

2. **pandas**:
   - `pd`: Biblioteca para la manipulación y análisis de datos. Proporciona estructuras de datos y herramientas para trabajar con datos estructurados (tablas).

3. **sklearn.datasets**:
   - `fetch_20newsgroups`: Función para cargar el conjunto de datos de 20 Newsgroups, utilizado para tareas de clasificación de texto.

4. **sklearn.feature_extraction.text**:
   - `TfidfVectorizer`: Clase para convertir una colección de documentos de texto en una matriz de características TF-IDF.

5. **sklearn.model_selection**:
   - `train_test_split`: Función para dividir un conjunto de datos en subconjuntos de entrenamiento y prueba.
   - `KFold`: Clase para la validación cruzada K-Fold.
   - `cross_val_score`: Función para evaluar un modelo mediante validación cruzada.

6. **sklearn.preprocessing**:
   - `LabelEncoder`: Clase para convertir etiquetas categóricas en valores numéricos.
   - `label_binarize`: Función para binarizar etiquetas.

7. **tensorflow.keras.utils**:
   - `to_categorical`: Función para convertir vectores de etiquetas en matrices de etiquetas categóricas (one-hot encoding).

8. **nltk**:
   - `stopwords`: Módulo que proporciona una lista de palabras comunes (stopwords) que se pueden excluir del análisis de texto.

9. **tensorflow.keras.models**:
   - `Sequential`: Clase para crear un modelo secuencial en Keras.

10. **tensorflow.keras.layers**:
    - `Dense`: Capa densa (totalmente conectada) en una red neuronal.
    - `Dropout`: Capa de regularización que aplica Dropout a la entrada.

11. **scikeras.wrappers**:
    - `KerasClassifier`: Wrapper para usar modelos de Keras con scikit-learn.

12. **plotly.express**:
    - `px`: Biblioteca para la creación de gráficos interactivos.

13. **plotly.graph_objects**:
    - `go`: Biblioteca para la creación de gráficos más complejos y personalizados.

14. **sklearn.metrics**:
    - `confusion_matrix`: Función para calcular la matriz de confusión.
    - `roc_curve`: Función para calcular la curva ROC.
    - `auc`: Función para calcular el área bajo la curva ROC.

Asegúrate de tener instaladas todas las librerías necesarias antes de ejecutar las celdas de código. Puedes instalar las librerías utilizando `pip` si aún no las tienes instaladas.

In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, label_binarize
from tensorflow.keras.utils import to_categorical
import nltk
from nltk.corpus import stopwords
# nltk.download('stopwords')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

from sklearn.metrics import confusion_matrix, roc_curve, auc

2024-09-05 18:11:49.029759: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-05 18:11:49.035718: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-05 18:11:49.092794: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-05 18:11:49.140390: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-05 18:11:49.197817: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

## Cargar el dataset

En esta sección, cargamos el conjunto de datos "The 20 Newsgroups" utilizando la función `fetch_20newsgroups` de la biblioteca `sklearn.datasets`. Este conjunto de datos contiene aproximadamente 20,000 documentos de noticias distribuidos en 20 categorías diferentes.

In [3]:
# Cargar el dataset

categories = None # Usar todas las categorías
newsgroups = fetch_20newsgroups(subset='all', categories=categories)

## Preprocesamiento de Texto

En esta sección, realizamos el preprocesamiento del texto y la preparación de los datos para el modelo de clasificación. Los pasos incluyen la eliminación de stopwords, la vectorización de texto utilizando TF-IDF, la codificación de etiquetas y la división del conjunto de datos en subconjuntos de entrenamiento y prueba.

In [4]:
# Preprocesamiento de texto
stopwords = set(stopwords.words('english'))

# Vectorizar con TF-IDF
vectorizer = TfidfVectorizer(stop_words=list(stopwords), max_features=5000)
X = vectorizer.fit_transform(newsgroups.data)

# Codificar las etiquetas
encoder = LabelEncoder()
y = encoder.fit_transform(newsgroups.target)
y = to_categorical(y)

# Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 44 stored elements and shape (1, 5000)>
  Coords	Values
  (0, 1565)	0.0657918248947064
  (0, 4323)	0.02321047654937476
  (0, 3226)	0.024132485748670034
  (0, 2667)	0.023279544387511123
  (0, 3096)	0.04294079126762866
  (0, 3463)	0.04164359127539171
  (0, 2212)	0.04268574426490302
  (0, 3465)	0.11654126811381199
  (0, 1224)	0.0964570380748921
  (0, 3831)	0.06779225045131826
  (0, 2580)	0.073361477358374
  (0, 3346)	0.17512470115927722
  (0, 1064)	0.17307482474417313
  (0, 402)	0.10938824966379977
  (0, 2062)	0.16465595732584834
  (0, 4005)	0.08332401352668063
  (0, 100)	0.08603730452246322
  (0, 1835)	0.13240084480317035
  (0, 4091)	0.09671864922135887
  (0, 1316)	0.09209555979526737
  (0, 2671)	0.18077491609307464
  (0, 1330)	0.10035021454829482
  (0, 2703)	0.30547755471161564
  (0, 2394)	0.08502642416641125
  (0, 2027)	0.12290023797769674
  (0, 4893)	0.2069088513022005
  (0, 3047)	0.10541014838068206
  (0, 1153)	0.205903862

## Visualización de la Distribución de Categorías y Longitud de Textos

En esta sección, visualizamos la distribución de las categorías en el conjunto de datos y la longitud de los textos utilizando gráficos interactivos. Esto nos permite entender mejor la estructura y características del conjunto de datos.

In [5]:
# Distribución de las categorías en el dataset
category_counts = np.bincount(newsgroups.target)
categories = newsgroups.target_names

# Histograma de categorías
fig = px.bar(x=categories, y=category_counts, labels={'x':'Category', 'y':'Count'}, title="Distribution of Categories")
fig.show()

# Diagrama de caja para la longitud de los textos
text_lengths = [len(text) for text in newsgroups.data]
fig_box = px.box(text_lengths, labels={'value':'Text Length'}, title="Distribution of Text Lengths")
fig_box.show()


## Definir Función para Crear el Modelo

En esta sección, definimos una función para crear un modelo de red neuronal utilizando `tensorflow.keras`. La función toma como argumento el tipo de función de pérdida a utilizar y devuelve un modelo compilado. La arquitectura del modelo es la siguiente:

- **Capa de Entrada**: 128 neuronas con activación ReLU.
- **Capa de Abandono**: Tasa de 50% para prevenir el sobreajuste, lo que ayuda a mejorar la generalización del modelo.
- **Capa Oculta 1**: 64 neuronas con activación ReLU, que permite capturar patrones complejos en los datos.
- **Capa de Salida**: 20 neuronas con activación softmax, adecuada para problemas de clasificación multiclase.


In [6]:
# Definir función para crear el modelo
def create_model(lossSelection):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(20, activation='softmax'))
    model.compile(optimizer='adam', loss=lossSelection, metrics=['accuracy'])
    return model

In [7]:
# Llamar al modelo
model = create_model("categorical_crossentropy")
# Resumen del modelo
model.summary()

/home/mtumalan/Desktop/Repos/Clases/ConcentracionIA/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │       640,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 20)             │         1,300 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 649,684 (2.48 MB)

 Trainable params: 649,684 (2.48 MB)

 Non-trainable params: 0 (0.00 B)

## Entrenamiento del Modelo

En esta sección, entrenamos el modelo de red neuronal utilizando los datos de entrenamiento. El proceso de entrenamiento se realiza durante un número especificado de épocas y con un tamaño de lote determinado. Además, se evalúa el rendimiento del modelo en los datos de prueba en cada época.

In [8]:
# Entrenamiento del modelo
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.3062 - loss: 2.5814 - val_accuracy: 0.8151 - val_loss: 0.8391
Epoch 2/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8302 - loss: 0.7198 - val_accuracy: 0.8637 - val_loss: 0.5004
Epoch 3/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9065 - loss: 0.3815 - val_accuracy: 0.8761 - val_loss: 0.4262
Epoch 4/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9428 - loss: 0.2418 - val_accuracy: 0.8820 - val_loss: 0.4006
Epoch 5/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9617 - loss: 0.1674 - val_accuracy: 0.8851 - val_loss: 0.4056
Epoch 6/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9716 - loss: 0.1253 - val_accuracy: 0.8846 - val_loss: 0.4097
Epoch 7/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9827 - loss: 0.0861 - val_accuracy: 0.8804 - val_loss: 0.4130
Epoch 8/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9849 - loss: 0.0725 - val_accuracy: 0.

## Visualización del Proceso de Entrenamiento

En esta sección, preparamos los datos del historial de entrenamiento para la visualización y creamos gráficos para mostrar la precisión y la pérdida tanto en el conjunto de entrenamiento como en el de validación a lo largo de las épocas.

In [9]:
# Preparar los datos para la visualización
history_df = pd.DataFrame({
    'Epoch': list(range(1, len(history.history['accuracy']) + 1)),
    'Training Accuracy': history.history['accuracy'],
    'Validation Accuracy': history.history['val_accuracy'],
    'Training Loss': history.history['loss'],
    'Validation Loss': history.history['val_loss']
})

# Graficar precisión
fig_acc = px.line(history_df, x='Epoch', y=['Training Accuracy', 'Validation Accuracy'], 
                  title="Training and Validation Accuracy")
fig_acc.show()

# Graficar pérdida
fig_loss = px.line(history_df, x='Epoch', y=['Training Loss', 'Validation Loss'], 
                   title="Training and Validation Loss")
fig_loss.show()

## Evaluación del Modelo y Visualización de la Matriz de Confusión

En esta sección, evaluamos el rendimiento del modelo en el conjunto de prueba y visualizamos la matriz de confusión utilizando un heatmap interactivo con Plotly. La matriz de confusión nos permite ver cómo se distribuyen las predicciones del modelo en comparación con las etiquetas verdaderas.

In [10]:
# Evaluar el conjunto de prueba para obtener predicciones
y_pred = np.argmax(model.predict(X_test), axis=1)  # Predicciones del modelo
y_true = np.argmax(y_test, axis=1)  # Etiquetas verdaderas

# Obtener la matriz de confusión
cm = confusion_matrix(y_true, y_pred)

# Crear el heatmap con Plotly
fig_cm = go.Figure(data=go.Heatmap(
    z=cm,
    x=categories,  # Nombres de las categorías en los ejes
    y=categories,
    colorscale='Blues',
    hoverongaps=False))

# Añadir detalles a la gráfica
fig_cm.update_layout(
    title='Confusion Matrix',
    xaxis_title='Predicted',
    yaxis_title='Actual',
    xaxis_nticks=36,  # Controla el número de divisiones en el eje
    yaxis_nticks=36,
)

# Mostrar el gráfico
fig_cm.show()

118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


# Creación de módelo con arquitectura alternativa

En la siguiente sección, crearemos un modelo con una arquitectura diferente, la cual será la siguiente:

- **Capa de Entrada**: 256 neuronas con activación ReLU.
- **Capa de Abandono**: Tasa de 50% para prevenir el sobreajuste.
- **Capa Oculta 1**: 128 neuronas con activación ReLU.
- **Capa Oculta 2**: 64 neuronas con activación ReLU.
- **Capa de Salida**: 20 neuronas con activación softmax, adecuada para problemas de clasificación multiclase.

In [11]:
# Crear el modelo alternativo
model_alt = Sequential()
model_alt.add(Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model_alt.add(Dropout(0.5))
model_alt.add(Dense(128, activation='relu'))
model_alt.add(Dense(64, activation='relu'))
model_alt.add(Dense(20, activation='softmax'))

# Compilar el modelo
model_alt.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history_alt = model_alt.fit(X_train, y_train)# Crear el modelo alternativo
model_alt = Sequential()
model_alt.add(Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model_alt.add(Dropout(0.5))
model_alt.add(Dense(128, activation='relu'))
model_alt.add(Dense(64, activation='relu'))
model_alt.add(Dense(20, activation='softmax'))

# Compilar el modelo
model_alt.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history_alt = model_alt.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# Convertir el historial de entrenamiento en un DataFrame
history_df = pd.DataFrame(history_alt.history)
history_df['epoch'] = history_df.index + 1  # Añadir la columna de épocas

# Visualización de la precisión con Plotly
fig_acc = go.Figure()

# Añadir la precisión del entrenamiento
fig_acc.add_trace(go.Scatter(
    x=history_df['epoch'],
    y=history_df['accuracy'],
    mode='lines',
    name='Training Accuracy'
))

# Añadir la precisión de la validación
fig_acc.add_trace(go.Scatter(
    x=history_df['epoch'],
    y=history_df['val_accuracy'],
    mode='lines',
    name='Validation Accuracy'
))

# Configuración del gráfico
fig_acc.update_layout(
    title='Training and Validation Accuracy',
    xaxis_title='Epoch',
    yaxis_title='Accuracy',
)

# Mostrar la gráfica de precisión
fig_acc.show()

# Visualización de la pérdida con Plotly
fig_loss = go.Figure()

# Añadir la pérdida del entrenamiento
fig_loss.add_trace(go.Scatter(
    x=history_df['epoch'],
    y=history_df['loss'],
    mode='lines',
    name='Training Loss'
))

# Añadir la pérdida de la validación
fig_loss.add_trace(go.Scatter(
    x=history_df['epoch'],
    y=history_df['val_loss'],
    mode='lines',
    name='Validation Loss'
))

# Configuración del gráfico
fig_loss.update_layout(
    title='Training and Validation Loss',
    xaxis_title='Epoch',
    yaxis_title='Loss',
)

# Mostrar la gráfica de pérdida
fig_loss.show()

/home/mtumalan/Desktop/Repos/Clases/ConcentracionIA/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



472/472 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.4148 - loss: 2.0392
Epoch 1/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.3598 - loss: 2.3069 - val_accuracy: 0.8369 - val_loss: 0.5595
Epoch 2/10
 46/236 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.8851 - loss: 0.4212

KeyboardInterrupt: 

In [ ]:
# Usar KerasClassifier y K-Fold cross-validation
estimator = KerasClassifier(build_fn=create_model("sparse_categorical_crossentropy"), epochs=10, batch_size=64, verbose=0)
kfold = KFold(n_splits=5, shuffle=True)
results = cross_val_score(estimator, X, newsgroups.target, cv=kfold)

print(f"K-Fold Cross-Validation Results: {results.mean():.2f} ± {results.std():.2f}")

/home/mtumalan/Desktop/Repos/Clases/ConcentracionIA/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

/home/mtumalan/Desktop/Repos/Clases/ConcentracionIA/venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:713: UserWarning:

Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 

/home/mtumalan/Desktop/Repos/Clases/ConcentracionIA/venv/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning:

``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.

/home/mtumalan/Desktop/Repos/Clases/ConcentracionIA/venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:713: UserWarning:

Skipping variable loading for optimizer 'ada

K-Fold Cross-Validation Results: 0.89 ± 0.00


In [ ]:
# Convertir las etiquetas a binario
y_bin = label_binarize(newsgroups.target, classes=range(20))
n_classes = y_bin.shape[1]

# Calcular la curva ROC para cada categoría
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_bin[:, i], model.predict(X)[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Graficar la curva ROC AUC
fig_roc = go.Figure()

# Añadir las curvas ROC para cada categoría
for i in range(n_classes):
    fig_roc.add_trace(go.Scatter( # Curva ROC para la categoría i
        x=fpr[i], y=tpr[i],
        mode='lines',
        name=f'{categories[i]} (AUC = {roc_auc[i]:.2f})'
    ))

# Configuración del gráfico
fig_roc.update_layout(
    title='ROC Curves for Each Category',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
)

# Mostrar la gráfica
fig_roc.show()

589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
